### Named Entity Recognition

References:

1. https://towardsdatascience.com/named-entity-recognition-and-classification-with-scikit-learn-f05372f07ba2
2. https://www.analyticsvidhya.com/blog/2018/08/nlp-guide-conditional-random-fields-text-classification/
3. https://www.kaggle.com/abhinavwalia95/entity-annotated-corpus

In [0]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
data_path = "/content/drive/My Drive/Datahack NLP Workshop/GMB_NER/"
df = pd.read_csv(data_path + "ner_dataset.csv", encoding = "ISO-8859-1")
df.shape

(1048575, 4)

In [0]:
df.head(15)

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O
5,NaN,through,IN,O
6,NaN,London,NNP,B-geo
7,NaN,to,TO,O
8,NaN,protest,VB,O
9,NaN,the,DT,O


In [0]:
df = df.fillna(method='ffill')
df = df[:50000]

In [0]:
df["Tag"].value_counts()

O        42547
B-geo     1490
B-gpe      968
B-org      959
I-per      931
B-tim      880
B-per      789
I-org      689
I-geo      303
I-tim      239
B-art       48
B-eve       39
I-eve       33
I-gpe       31
I-art       27
B-nat       18
I-nat        9
Name: Tag, dtype: int64

### Simple Model

In [0]:
X = df[["Word", "POS"]]
v = DictVectorizer(sparse=False)
X = v.fit_transform(X.to_dict('records'))
X.shape

(50000, 7504)

In [0]:
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [0]:
y = df.Tag.values
classes = np.unique(y).tolist()

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state=0)
X_train.shape, y_train.shape

((33500, 7504), (33500,))

In [0]:
clf = SGDClassifier(max_iter=5, random_state=2019)
clf.partial_fit(X_train, y_train, classes)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=5,
              n_iter_no_change=5, n_jobs=None, penalty='l2', power_t=0.5,
              random_state=2019, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [0]:
new_classes = [c for c in classes if c != "O"]
print(classification_report(y_pred=clf.predict(X_test), y_true=y_test, labels=new_classes))

              precision    recall  f1-score   support

       B-art       0.00      0.00      0.00        18
       B-eve       0.01      0.07      0.02        14
       B-geo       0.74      0.48      0.58       476
       B-gpe       0.94      0.45      0.61       320
       B-nat       0.00      0.00      0.00         3
       B-org       0.75      0.34      0.47       301
       B-per       0.87      0.40      0.54       258
       B-tim       0.91      0.72      0.81       272
       I-art       0.00      0.00      0.00         5
       I-eve       0.00      0.00      0.00        11
       I-geo       0.80      0.30      0.44       115
       I-gpe       0.00      0.00      0.00        11
       I-nat       0.00      0.00      0.00         3
       I-org       0.76      0.28      0.41       200
       I-per       0.28      0.97      0.44       306
       I-tim       0.00      0.00      0.00        90

   micro avg       0.54      0.48      0.51      2403
   macro avg       0.38   

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### NER using modules

##### Flair

https://github.com/zalandoresearch/flair/blob/master/resources/docs/TUTORIAL_2_TAGGING.md

In [0]:
!pip install flair

In [0]:
from flair.models import SequenceTagger

tagger = SequenceTagger.load('ner')

2019-11-23 05:21:48,907 loading file /root/.flair/models/en-ner-conll03-v0.4.pt


In [0]:
from flair.data import Sentence
sentence = Sentence('George Washington went to Washington .')

# predict NER tags
tagger.predict(sentence)

# print sentence with predicted tags
print(sentence.to_tagged_string())

George <B-PER> Washington <E-PER> went to Washington <S-LOC> .


##### Spacy

https://spacy.io/usage/linguistic-features#named-entities

In [0]:
import spacy

nlp = spacy.load("en_core_web_sm")

In [0]:
doc = nlp("George Washington went to Washington .")

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

George Washington 0 17 PERSON
Washington 26 36 GPE
